In [14]:
import pandas as pd
import numpy as np
import altair as alt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error

In [5]:
pd.options.display.float_format='{:,.2f}'.format
import warnings
warnings.simplefilter("ignore")

In [75]:
df= pd.read_csv('PIB_Ecuador.csv',sep=',')
df.head()

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
0,Aruba,ABW,PIB per cápita (UMN actual),NY.GDP.PCAP.CN,nan,nan,nan,nan,nan,nan,...,"44,724.93","44,237.52","46,879.09","47,699.81","50,085.78","50,623.62","51,923.77",nan,nan,nan
1,Afganistán,AFG,PIB per cápita (UMN actual),NY.GDP.PCAP.CN,"2,689.79","2,693.74","2,630.61","3,541.79","3,694.29","4,549.87",...,"28,234.41","33,015.04","35,290.50","35,254.46","35,531.20","34,562.15","35,415.64","35,717.12","38,631.14",nan
2,Angola,AGO,PIB per cápita (UMN actual),NY.GDP.PCAP.CN,nan,nan,nan,nan,nan,nan,...,"433,553.07","486,895.95","507,192.31","531,659.73","500,290.50","573,791.27","679,560.90","831,805.91","1,018,129.07",nan
3,Albania,ALB,PIB per cápita (UMN actual),NY.GDP.PCAP.CN,nan,nan,nan,nan,nan,nan,...,"447,689.08","459,526.45","466,324.61","482,954.10","497,901.56","511,970.59","539,644.58","570,655.99","588,058.23",nan
4,Andorra,AND,PIB per cápita (UMN actual),NY.GDP.PCAP.CN,nan,nan,nan,nan,nan,nan,...,"31,132.10","30,109.67","29,768.74","31,089.47","32,232.76","33,854.61","34,489.94","35,390.36","36,523.81",nan


In [77]:

df = df[df['Country Name']=='Ecuador']


In [78]:
df.head()


,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2011,2012,2013,2014,2015,2016,2017,2018,2019,2020
64,Ecuador,ECU,PIB per cápita (UMN actual),NY.GDP.PCAP.CN,379.37,343.79,315.54,368.45,440.35,454.98,...,"5,200.56","5,682.05","6,056.33","6,377.09","6,124.49","6,060.09","6,213.50","6,295.94","6,183.82",nan


In [79]:
df = df[df['Country Name']=='Ecuador']
df = pd.DataFrame(df.iloc[0].loc['1960':])

In [80]:
df.head()

,64
1960,379.37
1961,343.79
1962,315.54
1963,368.45
1964,440.35


In [81]:
df.reset_index(inplace=True)
df.columns = ['year','pib_value']

In [82]:
df.head()

,year,pib_value
0,1960,379.37
1,1961,343.79
2,1962,315.54
3,1963,368.45
4,1964,440.35


In [83]:
alt.Chart(df.melt('year')).mark_line().encode(
x='year',
y=alt.Y('value',title='PIB value $'),
color='variable'
).properties(title='PIB Ecuador')

alt.Chart(...)

In [84]:
df['change']=df['pib_value'].diff()
df['change']=(df['change']/df['pib_value']) *100
df=df.fillna(0)
df.head()

,year,pib_value,change
0,1960,379.37,0.00
1,1961,343.79,-10.35
2,1962,315.54,-8.95
3,1963,368.45,14.36
4,1964,440.35,16.33


In [85]:
alt.Chart(df.drop('pib_value',axis=1).melt('year')).mark_line().encode(
x='year',
y=alt.Y('value',title='PIB Change %'),
color='variable'
).properties(title='PIB Ecuador Change')

alt.Chart(...)

In [86]:
print("PIB más alto es :")
pd.DataFrame(df.max()).T

PIB más alto es :


,year,pib_value,change
0,"2,020.00","6,377.09",39.35


In [87]:
print("PIB más bajo es :")
pd.DataFrame(df.min()).T

PIB más bajo es :


,year,pib_value,change
0,"1,960.00",0.00,-45.28


In [88]:
print("PIB más alto es :")
df[df['change']==pd.DataFrame(df.max()).T['change'][0]]

PIB más alto es :


,year,pib_value,change
14,1974,969.55,39.35


In [89]:
dict_n_vs_p = {'positive':df.query('change>0').count()[0],'negative':df.query('change<0').count()[0]}
df_pn = pd.DataFrame(dict_n_vs_p,index=[0]).T
df_pn.reset_index(inplace=True)
df_pn.columns = ['description','value']

In [90]:
alt.Chart(df_pn).mark_bar(size=30).encode(
x='description',
y=alt.Y('value',title='count values'),
color='description'
).properties(width=alt.Step(100),title='Positive vs Negative PIB')

alt.Chart(...)

In [91]:
data=pd.DataFrame(df['pib_value'].diff())
data=data.fillna(0.00)
dict_n_vs_p = {'positive':data.query('pib_value>0').sum()[0],'negative':np.abs(data.query('pib_value<0').sum()[0])}
df_pn = pd.DataFrame(dict_n_vs_p,index=[0]).T
df_pn.reset_index(inplace=True)
df_pn.columns = ['description','value']

In [92]:
alt.Chart(df_pn).mark_bar(size=30).encode(
x='description',
y=alt.Y('value',title='value PIB $'),
color='description'
).properties(width=alt.Step(150),title='Positive vs Negative PIB affection')

alt.Chart(...)

In [99]:
def _obtain_feature_target(df,feature_columns,target_columns):
    features = df[feature_columns]
    target= df[target_columns]
    return features,target


In [100]:
features,target = _obtain_feature_target(df,['year'],'pib_value')

In [101]:
model_linear = LinearRegression()
model_linear = model_linear.fit(features,target)
print("Score linear model = {}".format(model_linear.score(features,target)))

Score linear model = 0.6709336527848547


In [103]:
predict_linear = pd.DataFrame({'year':features.values.reshape(-1),'real':target.values,'predict':model_linear.predict(features)})

In [104]:
alt.Chart(predict_linear.melt('year')).mark_line().encode(
x='year',
y='value',
color='variable'
).properties(title='Linear model predictions')

alt.Chart(...)

In [105]:
model_poly = Pipeline([('poly', PolynomialFeatures(degree=3)),
('linear', LinearRegression(fit_intercept=True))])
model_poly = model_poly.fit(features,target)
print("Score polinomyal model = {}".format(model_poly.score(features,target)))

Score polinomyal model = 0.7134725577933035


In [107]:
predict_polinomyal = pd.DataFrame({'year':features.values.reshape(-1),'real':target.values,'predict':model_poly.predict(features)})

In [108]:
alt.Chart(predict_polinomyal.melt('year')).mark_line().encode(
x='year',
y='value',
color='variable'
).properties(title='polinomyal model predictions')

alt.Chart(...)

In [115]:
predictions=pd.DataFrame({'year':[i for i in range(2020,2026)]})
df_predict = pd.DataFrame({'year':['{}'.format(i) for i in range(2020,2026)],
    'linear':model_linear.predict(predictions),
    'poly':model_poly.predict(predictions)})

In [116]:
df_predict

,year,linear,poly
0,2020,"4,876.90","5,897.90"
1,2021,"4,962.86","6,116.56"
2,2022,"5,048.81","6,342.18"
3,2023,"5,134.76","6,574.90"
4,2024,"5,220.71","6,814.86"
5,2025,"5,306.66","7,062.20"


In [120]:
alt.Chart(df_predict.melt('year')).mark_point().encode(
x='year',
y='value',
color='variable'
)+alt.Chart(df_predict.melt('year')).mark_line().encode(
x='year',
y='value',
color='variable'
).properties(title='Prediction from 2020 to 2025',width=alt.Step(70))

alt.LayerChart(...)